In [ ]:
from IPython.display import display, Math, Latex, Markdown
from myst_nb import glue

(vmm)=
# Vessel Manoeuvring Model Models

Ship manoeuvring is a simplified case of seakeeping. The encountering waves have been removed, assuming calm water conditions. This simplification allows for the ship dynamics to be expressed with only four degrees of freedom: surge, sway, roll and yaw, where roll is often excluded. Surge, sway and yaw have very low frequencies during manoeuvres, so that added masses as well as other hydrodynamic derivatives can be assumed as constants $\cite{fossen_handbook_2021}$. Three Vessel Manoeuvring Models (VMMs) are used in this paper: Linear (LVMM) $\cite{matusiak_dynamics_2017}$, Abkowitz model (AVMM) $\cite{abkowitz_ship_1964}$ and a Modified Abkowitz model (MAVMM) proposed in this study.

{numref}`coordinate_system` shows the coordinate systems used in the VMMs where $x_0$ and $y_0$ and heading $\Psi$ are the global position and orientation of a ship fix coordinate system $O(x,y,z)$,  with origin at midship. $u$, $v$, $r$, $X$, $Y$ and $N$ are velocities and forces in the ship fix coordinate system.

```{figure} figures/coordinate_system.PNG
---
height: 300px
name: coordinate_system
---
Coordinate system
```

(vmm)=
# Models
Three Vessel Manoeuvring Models (VMMs) are used in this paper: Linear (LVMM), {cite:p}`matusiak_dynamics_2017`, Abkowitz model (AVMM) $\cite{abkowitz_ship_1964}$ and a Modified Abkowitz model (MAVMM) which is  proposed by the authors. These VMMs and almost all other VMM:s that exist have a very similar structure. Ship manoeuvring is a simplified case of seakeeping. The encountering waves have been removed, assuming calm water conditions. This simplification allows for the ship dynamics to be expressed with only four degrees of freedom: surge, sway, roll and yaw, where roll is often excluded. Surge, sway and yaw have very low frequencies during manoeuvres, so that added masses as well as other hydrodynamic derivatives can be assumed as constants $\cite{fossen_handbook_2021}$.

{numref}`coordinate_system` shows the coordinate systems used in the VMMs where $x_0$ and $y_0$ and heading $\Psi$ are the global position and orientation of a ship fix coordinate system $O(x,y,z)$,  with origo at midship. $u$, $v$, $r$, $X$, $Y$ and $N$ are velocities and forces in the ship fix coordinate system.

```{figure} figures/coordinate_system.PNG
---
height: 300px
name: coordinate_system
---
Coordinate system
```

![](figures/coordinate_system.PNG)

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

vmms = {}
for vmm_name in vmm_names:
    vmms[vmm_name] = catalog.load(vmm_name)


The vessel manoeuvring models can be expressed in a very general way ( $\cite{fossen_handbook_2021}$):
```{glue:math} eq6dof
:label: eq6dof
```

In [ ]:
glue("eq6dof", eq_6DOF)

Where $\eta$ describes the position:
```{glue:math} eqeta
:label: eqeta
```

In [ ]:
glue("eqeta", eq_eta)

and $\nu$ is the velocities:
```{glue:math} eqnu
:label: eqnu
```

In [ ]:
glue("eqnu", eq_nu)

The accelerations are denoted using the dotted notation: $\dot{\nu}$.

* $M$ is inertia matrix
* $C(\nu)$ is corriolis/centrepetal matrix as function of the velocities \nu.
* $D(\nu)$ is damping matrix as a function of vecocities \nu.
* $g(\eta)$ is a vector of generalized gravitational an buoyance forces.
* $g_0$ is static restoring forces due to ballast systems.
* $\tau$ is vector of control inputs (from rudders/propellers etc.)
* $\tau_{wind}$ is vector of wind forces
* $\tau_{wave}$ is vector of wave forces

The velocities can also include the ocean current by expressing the relative velocity $v_r$ as:
```{glue:math} eqnur
:label: eqnur
```

In [ ]:
glue("eqnur", eq_nu_r)

If the current is assumed to be irrotational, the angular velocities for the current is zero:
```{glue:math} eqnuc
:label: eqnuc
```

```{glue:math} eq_nu_c
:label: eq_nu_c
```

```{glue:math} eqnurexpanded
:label: eqnurexpanded
```

In [ ]:
glue("eqnuc", eq_nu_c)

In [ ]:
glue("eqnurexpanded", eq_nu_r_expanded)

If the current is also assumed to be constant, this mean that the time derivative of $\nu$ and $\nu_r$ are the same, also giving the same accelerations: 
```{glue:math} eqnusteady
:label: eqnusteady
```

In [ ]:
glue("eqnusteady", eq_nu_steady)

When current is present the inertia as well as the corriolis matrix must be split into an added mass part (A) and a rigid body part (RB):
```{glue:math} eqm
:label: eqm
```

```{glue:math} eqc
:label: eqc
```

In [ ]:
glue("eqm", eq_M) 

In [ ]:
glue("eqc", eq_C) 

So that the model equation can be written as:

```{glue:math} eq6DOFexpanded
:label: eq6DOFexpanded
```

In [ ]:
glue("eq6DOFexpanded", eq_6DOF_expanded) 

This equation can be simplified for manoeuvring models by firstly only keeping surge, sway and yaw degrees of freedome. This means that both $g_0$ and $g(\eta)$ dissapears as there are no static forces for surge, sway and yaw. For the classic manoeuvring problem also forces from wind and waves are neglected, removing $\tau_{wave}$ and $\tau_{wind}$. In this paper, there are no ocean current during the studied model tests, so that the relative velocity $v_r$ can be replaced with $v$.

```{glue:math} eq3DOF
:label: eq3DOF
```

In [ ]:
eq_3DOF = eq_6DOF_expanded.subs([

    (g_0,0),
    (g_function, 0),
    (tau_wave,0),
    (tau_wind,0),
    (nu_r,nu)

])

glue("eq3DOF", eq_3DOF) 



And these matrices can be expressed in 3 degrees of freeddome ( {cite:p}`fossen_handbook_2021`):

```{glue:math} eqCA
:label: eqCA
```

```{glue:math} eq_C_RB
:label: eq_C_RB
```

```{glue:math} eq_M_A
:label: eq_M_A
```

```{glue:math} eq_M_RB
:label: eq_M_RB
```

In [ ]:
glue("eqCA", eq_C_A) 

In [ ]:
glue("eq_C_RB", eq_C_RB) 

In [ ]:
glue("eq_M_A", eq_M_A) 

In [ ]:
glue("eq_M_RB", eq_M_RB) 

The damping and control inputs from rudders and propellers are replaced with the functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$. Note that the measured thrust from the model tests is used as input to the models, which means that the propeller is not part of the models. Main focus is thereby on the modelling of rudder and hull forces. A separate propeller model is needed to provide the thrust input to this model.

```{glue:math} eq_D_function
:label: eq_D_function
```

In [ ]:
glue("eq_D_function", eq_D_function)  

In [ ]:
CD_ = (eq_C_A.rhs + eq_C_RB.rhs).doit()

```{glue:math} eqqsystem
:label: "eqqsystem"
```

The above equation ([eq](eqqsystem)) $\cite{fossen_handbook_2021}$ can be rewritten to get the acceleration on the left hand side ([eq](eqacc)),


```{glue:math} eqacc
:label: "eqacc"
```

In [ ]:
glue("eqqsystem", eq_system) 

In [ ]:
glue("eqacc", eq_acceleration_matrix_clean) 

where $S$ is a helper variable:
```{glue:math} eq_S
:label: eq_S
```

In [ ]:
glue("eq_S", eq_S)  

A state space model for manoeuvring can now be defined with six states:

```{glue:math} eq_x
:label: eq_x
```

In [ ]:
glue("eq_x", eq_x)  

The time derivative of this state $\dot{\mathbf{x}}$ can be defined by a state transition $f(\mathbf{x},\mathbf{c})$ using geometrical relations
how global coordinates $x_0$, $y_0$ and $\Psi$ depend on $u$, $v$, and $r$ viz.,

```{glue:math} eq_f
:label: "eqf"
```

In [ ]:
eq_transition = sp.Eq(x1d, eq_f.rhs)

eq_transition = Math(r'\dot{\mathbf{x}} = f(\mathbf{x},\mathbf{c}) + \mathbf{w} = \left[\begin{matrix}u \cos{\left(\Psi \right)} - v \sin{\left(\Psi \right)}\\u \sin{\left(\Psi \right)} + v \cos{\left(\Psi \right)}\\r\\\dot{u}\\\dot{v}\\\dot{r}\end{matrix}\right] + \mathbf{w}')

glue("eq_f", eq_transition)

where $\mathbf{c}$ is control inputs (rudder angle $\delta$ and thrust $T$); the last three derivatives: $\dot{u}$, $\dot{v}$, $\dot{r}$ are calculated with [eq](eqacc).
$\mathbf{w}$ is the process noise, i.e., the difference between the predicted state by the VMM and the true
state of the system. $\mathbf{w}$ is unknown when the VMM is used for manoeuvre predictions and therefore normally
assumed to be zero, but it is an important factor when the VMM is used in the EKF, see Section $\ref{\detokenize{04.01_EK::doc}}$.

The manoeuvring simulation can now be conducted by numerical integration of [eq](eqf). The main difference between the VMM:s lies in how the hydrodynamic functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$ are defined. These expressions are denoted for different VMMs, namely LVMM, AVMM and MAVMM below,

In [ ]:
from wPCC_pipeline.jupyter_book import multiline

primes = ["u","v","r","thrust"]
subs_prime={sp.symbols(symbol):sp.symbols(f"{symbol}'") for symbol in primes}

for vmm_name, vmm in vmms.items():
    
    #appendix.add_header(labels.get(vmm_name), level=3)
    eqs = {'X_D':vmm.X_qs_eq.subs(subs_prime),
           'Y_D':vmm.Y_qs_eq.subs(subs_prime),
           'N_D':vmm.N_qs_eq.subs(subs_prime),
          }
    
    for name, eq in eqs.items():
        label = f"eq{name[0:-2]}{vmm_name[4:]}"
        s_latex = multiline(eq, terms_per_line=8)
        for symbol in ["X_{D}","Y_{D}","N_{D}"]:
            s_latex = s_latex.replace(symbol,f"{symbol}'")
        
        glue(label, Latex(s_latex), display=False)
        #glue(label, eq, display=False)
        s = f"""
```{{glue:math}} {label}
:label: {label.lower().replace('_','')}

```"""
        print(s)
        
        
        

LVMM (Linear Vessel Manoeuvring Model) $\cite{matusiak_dynamics_2017}$:

```{glue:math} eqXlinear
:label: eqxlinear

```


```{glue:math} eqYlinear
:label: eqylinear

```


```{glue:math} eqNlinear
:label: eqnlinear

```




AVMM (Abkowitz Vessel Manoeuvring Model) $\cite{abkowitz_ship_1964}$:

```{glue:math} eqXabkowitz
:label: eqxabkowitz

```


```{glue:math} eqYabkowitz
:label: eqyabkowitz

```


```{glue:math} eqNabkowitz
:label: eqnabkowitz

```


MAVMM (Modified Abkowitz Vessel Manoeuvring Model where only the most relevant coefficients in AVMM are included.


```{glue:math} eqXmartins_simple
:label: eqxmartinssimple

```


```{glue:math} eqYmartins_simple
:label: eqymartinssimple

```


```{glue:math} eqNmartins_simple
:label: eqnmartinssimple

```






In [ ]:
from wPCC_pipeline.jupyter_book import Appendix
appendix = Appendix(file_path='appendix_vmms.md', title='Vessel Manoeuvring Models')

labels = {

    'vmm_linear':'Linear model',
    'vmm_abkowitz':'MAVMM', 
    'vmm_martins_simple': 'MAVMM'

}

for vmm_name, vmm in vmms.items():
    
    appendix.add_header(labels.get(vmm_name), level=3)
    eqs = {'X_D':vmm.X_qs_eq,
           'Y_D':vmm.Y_qs_eq,
           'N_D':vmm.N_qs_eq,
          }
    
    for name, eq in eqs.items():
        label = f"eq_{name}_{vmm_name}"
        appendix.add_equation_multiline(eq=eq, label=label)

In [ ]:
from src.prime_system import df_prime
from jb_helpers import df_to_myst

denominators = df_prime.loc['denominator'].copy()
denominators.drop(index=['-'], inplace=True)
index = denominators.index
prime_table = pd.DataFrame(index=index)
prime_table['Denominators'] = denominators.apply(lambda x : f"${sp.latex(x)}$")
prime_table.index.name = 'Physical quantity'
prime_table.index = [item.replace('_','\n') for item in prime_table.index]

print(df_to_myst(prime_table, title='Prime system denominators'))

The hydrodynamic functions above are expressed with nondimensional prime units denoted with the prime symbol ('). The quantities are expressed in the prime system, using the denominators in {numref}`Prime_system_denominators`. Surge linear velocity $u$ can for instance be expressed in prime system as seen in [eq](eqprime) using the linear velocity denominator.

```{glue:math} eqprime
:label: eqprime
```

In [ ]:
eqprime = Math(r"$u'=\frac{u}{U}$")
glue("eqprime", eqprime)

```{list-table} Prime system denominators
:header-rows: 1
:name: Prime_system_denominators
* -  
  - Denominators
* - length
  - $L$
* - volume
  - $L^{3}$
* - mass
  - $\frac{L^{3} \rho}{2}$
* - density
  - $\frac{\rho}{2}$
* - inertia
moment
  - $\frac{L^{5} \rho}{2}$
* - time
  - $\frac{L}{U}$
* - frequency
  - $\frac{U}{L}$
* - area
  - $L^{2}$
* - angle
  - $1$
* - linear
velocity
  - $U$
* - angular
velocity
  - $\frac{U}{L}$
* - linear
acceleration
  - $\frac{U^{2}}{L}$
* - angular
acceleration
  - $\frac{U^{2}}{L^{2}}$
* - force
  - $\frac{L^{2} U^{2} \rho}{2}$
* - moment
  - $\frac{L^{3} U^{2} \rho}{2}$

```